In [ ]:
from pathlib import Path
import re

log_paths = [Path(d, 'checkpoint.log') for d in Path('/Users/marwei/AFS/home/urz/m/marwei/Logs/IDEAL/cifar100_fixed_slots').glob('cifar100_*')] \
          + [Path(d, 'checkpoint.log') for d in Path('/Users/marwei/AFS/home/urz/m/marwei/Logs/IDEAL/cifar10_fixed_slots').glob('cifar10_*')]

c = []
for this_dir in log_paths:
    with open(this_dir) as infile:
        loglines = infile.read().splitlines()

    mem_size = int(re.findall(r"memory_size=(\d+)", loglines[0])[0])
    final_acc = float(re.findall(r"Acc: \[(.*?)\]", loglines[-1])[0])
    compressor = re.findall(r"compressor=\'(.*?)\'", loglines[0])[0]
    encoder = re.findall(r"encoder=\'(.*?)\'", loglines[0])[0]
    dataset = re.findall(r"dataset=\'(.*?)\'", loglines[0])[0]

    compressor_param = None

    if compressor == 'autoencoder':
        compressor = 'convae'

    if compressor == 'thinning':
        compressor_param = float(re.findall(r"compression_factor=(.*?),", loglines[0])[0])
    elif compressor == 'quantization':
        compressor_param = int(re.findall(r"n_states=(\d+)", loglines[0])[0])
        try:
            strategy = re.findall(r"strategy=\'(.*?)\'", loglines[0])[0]
            assert strategy == 'tiny_imagenet_transfer' or strategy == 'local'
            if strategy == 'tiny_imagenet_transfer':
                compressor_new = 'quantization transfer'
            elif strategy == 'local':
                compressor_new = 'quantization local'
            else:
                raise ValueError('Unknown Quantization Stragegy: ' + strategy)
        except IndexError:
            compressor_new = 'quantization local'
        compressor = compressor_new
    elif compressor == 'convae':
        compressor_param = int(re.findall(r"latent_channels=(\d+)", loglines[0])[0])
    elif compressor == 'fcae':
        compressor_param = int(re.findall(r"bottleneck_neurons=(\d+)", loglines[0])[0])
    elif compressor == 'none':
        compressor_param = 0
    else:
        raise ValueError(f'Unknown Compressor: {compressor}')

    encoder = 'cutr' if encoder == 'cutr34' else encoder

    c.append({
        'mem_size': mem_size,
        'final_acc': final_acc,
        'encoder': encoder,
        'compressor': compressor,
        'compressor_param': compressor_param,
        'dataset': dataset
    })

In [ ]:
c

In [ ]:
import pandas as pd

df = pd.DataFrame.from_records(c)

In [ ]:
view = df.loc[
    (df.compressor != 'none')
    & (df.compressor != 'quantization local')
    & (df.compressor != 'fcae')
].copy()

compressor_names = {
    'thinning': 'Thinning',
    'quantization transfer': 'Quantization',
    'convae': 'Autoencoder'
}

encoder_names = {
    'cutr': 'FETCH',
    'none': 'no Fixed Encoder'
}

view.sort_values('compressor_param', inplace=True)

view['compressor_name'] = view['compressor'].map(compressor_names)
view['encoder_name'] = view['encoder'].map(encoder_names)


In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from plot_utils import science_template

fig = px.line(
    view,
    x='compressor_param',
    y='final_acc',
    color='encoder_name',
    facet_col='compressor_name',
    facet_row='dataset',
    template=science_template,
    markers=True,
    labels={
        'final_acc': 'Accuracy',
        'compressor_param': 'k',
        'encoder': 'Encoder',
    },
    category_orders={
        'compressor': ['thinning', 'quantization transfer', 'convae'],
        'dataset': ['CIFAR10', 'CIFAR100']
    }
)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_xaxes(matches=None, rangemode="tozero")
fig.update_yaxes(rangemode="tozero")
fig.update_layout(legend={'title_text':''})

# add baselines
for this_idx, this_dataset in enumerate(view['dataset'].unique()):
    y_gdumb = df.loc[(df['encoder']=='none') & (df['compressor'] == 'none') & (df['dataset'] == this_dataset), 'final_acc'].item()
    y_cutr3 = df.loc[(df['encoder']=='cutr') & (df['compressor'] == 'none') & (df['dataset'] == this_dataset), 'final_acc'].item()

    y_random = 0.1 if this_dataset=='CIFAR10' else 0.01

    fig.add_hline(y=y_gdumb,
                line_dash="solid",
                line_width=1,
                annotation_text="",
                annotation_position="top right",
                row=this_idx,
                col="all"
                )
    fig.add_hline(y=y_cutr3,
                line_dash="dot",
                line_width=1,
                annotation_text="",
                annotation_position="bottom right",
                row=this_idx,
                col="all"
                )
    fig.add_hline(y=y_random,
                line_dash="dash",
                line_width=1,
                annotation_text="",
                annotation_position="bottom right",
                row=this_idx,
                col="all"
                )

# legend below plot
fig.update_layout(
    font_size=16
)
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="top",
    y=-0.2,
    xanchor="left",
    x=0,
    font_size=15
))

# add invisible data so the baseline-lines appear in the legend
fig.add_traces(
    [
        go.Scatter(
            x=[fig.data[0].x[0]],
            y=[fig.data[0].y[0]], 
            showlegend=True,
            name='GDumb',
            mode='lines',
            line_dash='solid',
            line_color='black',
            line_width=1
        ),
        go.Scatter(
            x=[fig.data[0].x[0]],
            y=[fig.data[0].y[0]], 
            showlegend=True,
            name='Fixed Encoder, no Compressor',
            mode='lines',
            line_dash='dot',
            line_color='black',
            line_width=1
        ),
        go.Scatter(
            x=[fig.data[0].x[0]],
            y=[fig.data[0].y[0]], 
            showlegend=True,
            name='random guessing',
            mode='lines',
            line_dash='dash',
            line_color='black',
            line_width=1
        ),
    ]
)

for i in range(1, 4):
    fig.update_xaxes(
        title_standoff=5,
        row=1,
        col=i,
    )

fig.layout.yaxis.matches = 'y'
fig.layout.yaxis2.matches = 'y'
fig.layout.yaxis3.matches = 'y'
fig.layout.yaxis4.matches = 'y4'
fig.layout.yaxis5.matches = 'y4'
fig.layout.yaxis6.matches = 'y4'


In [ ]:
width = 550
height = 450

config = {
    'displaylogo': False,
    'toImageButtonOptions': {
        'format': 'svg', # one of png, svg, jpeg, webp
        'filename': 'plot',
        'height': height,
        'width': width,
        'scale': 5 # Multiply title/legend/axis/canvas sizes by this factor
    }
}

fig.show(config=config)
fig.write_image('../plots/eoc.pdf', width=width, height=height)

* due to a bug in plotly part of the legend is cut of when exporting the figure
* workaround
    * print the figure as pdf
    * use `pdfCropMargins`(availabe via pip)